In [12]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from sklearn import feature_extraction, linear_model, model_selection, metrics
from sklearn import ensemble
from scipy import sparse
import sys
sys.path.append("..")
from src.make_model_lstm import hate_speech_model

In [25]:
#watch overfitting


In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
df = pd.read_csv('../data/labeled_data.csv')

In [9]:
tweets = df['tweet'].apply(lambda x: x.lstrip('!'))

In [10]:
y = np.argmax(df[['hate_speech', 'offensive_language', 'neither']].values, axis = 1)

In [7]:
stop_words = set(stopwords.words('english')) 

In [8]:
token = nltk.tokenize.casual.TweetTokenizer()

In [13]:
lemmatizer = WordNetLemmatizer()

In [43]:
cleaned = []
wordDict = {}
i = 1
for tweet in tweets:
    tokenized = token.tokenize(tweet)
    newSent = []
    for word in tokenized:
        if word not in stop_words:
            newWord = lemmatizer.lemmatize(word)
            if newWord not in wordDict:
                wordDict[newWord] = i
                i += 1
            newSent.append(newWord)
            
    cleaned.append(newSent)

In [44]:
print(cleaned[0])
print(len(cleaned))

['RT', '@mayasolovely', ':', 'As', 'woman', 'complain', 'cleaning', 'house', '.', '&', 'man', 'always', 'take', 'trash', '...']
24783


In [45]:
maxWords = max(map(len, cleaned))

In [46]:
def create_seq(sent, vocab, maxWords):
    n = len(sent)
    numZeros = maxWords - n
    result = [0]*numZeros
    
    for word in sent:
        if word in vocab:
            result.append(vocab[word] + 2)
        else:
            result.append(1)
            
    return result

In [47]:
sequences = np.array(list(map(lambda x: create_seq(x, wordDict, maxWords), cleaned)))

In [48]:
sequences[0]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17])

In [51]:
y_cat = np.zeros((len(y), 3))
for i, l in enumerate(y):
    y_cat[i, l] = 1

In [52]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(sequences, y_cat, test_size = .1)

In [61]:
print(max(wordDict.values()))

41243


In [69]:
model = hate_speech_model()
model.build_model()
model.fit(X_train, y_train, epochs = 3)
preds = model.predict(X_test)

>> Compiled...
Train on 21188 samples, validate on 1116 samples
Epoch 1/3
21188/21188 [==============================] - 60s 3ms/sample - loss: 0.5791 - val_loss: 0.3730
Epoch 2/3
21188/21188 [==============================] - 57s 3ms/sample - loss: 0.2822 - val_loss: 0.3303
Epoch 3/3
21188/21188 [==============================] - 57s 3ms/sample - loss: 0.1713 - val_loss: 0.3560


In [73]:
print(metrics.roc_auc_score(y_test, preds))

0.9100288670230242


In [74]:
y_true = np.argmax(y_test, axis = 1)
y_pred = np.argmax(preds, axis = 1)

In [75]:
print(metrics.classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.37      0.17      0.24       145
           1       0.90      0.95      0.92      1922
           2       0.81      0.75      0.78       412

    accuracy                           0.87      2479
   macro avg       0.69      0.62      0.65      2479
weighted avg       0.85      0.87      0.86      2479



In [76]:
print(metrics.confusion_matrix(y_true, y_pred))

[[  25  105   15]
 [  37 1828   57]
 [   5  100  307]]
